In [6]:
#Jai Ganesh Deva

In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize  import word_tokenize
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline,metrics, grid_search
from gensim import models, corpora
from gensim.models import ldamodel
import os
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import io
import keras.backend as K
import csv
import numpy as np
import pandas as pd
import dill as pickle
import gensim
from keras.optimizers import Adam
from gensim.models import word2vec,doc2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn import metrics

C:\Users\vamsi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\vamsi\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [8]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import  Dense, Input,Conv1D,Conv2D,Convolution1D,GRU,Convolution2D ,LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, Masking,MaxPooling1D,GlobalMaxPool1D,Bidirectional,Merge,Flatten,SpatialDropout1D
from keras.models import Model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import gensim.models.keyedvectors as word2vec
import gc

In [9]:
os.chdir("E:\Datasets\Final Project\Politics Final\Big")

In [10]:
X_train = pd.read_csv("train_Bigdf.csv")

In [11]:
X_train = X_train.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis = 1)

In [12]:
X_test = pd.read_csv("test_Bigdf.csv")

In [13]:
X_test = X_test.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis = 1)

In [14]:
train_Y = np.array(X_train["sarcastic or not"])

In [15]:
test_Y = np.array(X_test["sarcastic or not"])

In [16]:
list_sentences_train = X_train["lower_wp"]
list_sentences_test = X_test["lower_wp"]

In [17]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
   
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
   
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [19]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features,char_level=True)

In [20]:
tokenizer.fit_on_texts(list(X_train["lower_wp"]))

In [21]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_sentences_test = tokenizer.texts_to_sequences(list_sentences_test)

In [22]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_sentences_test, maxlen=maxlen)



In [39]:
inp = Input(shape=(maxlen, ))

In [40]:
embed_size = 200
x = Embedding(len(tokenizer.word_index)+1, embed_size)(inp)

In [41]:
x = Conv1D(filters=256,kernel_size=4,padding='same', activation='relu')(x)

In [42]:
x=MaxPooling1D(pool_size=2)(x)

In [43]:
x = Bidirectional(GRU(256, return_sequences=True,name='lstm_layer',dropout=0.2,recurrent_dropout=0.2))(x)

In [44]:
x = GlobalMaxPool1D()(x)

In [45]:
x = Dense(50, activation="relu")(x)

In [46]:
x = Dropout(0.8)(x)
x = Dense(1, activation="sigmoid")(x)

In [47]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                 metrics=['accuracy',f1])

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 200)          7800      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 256)          205056    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 100, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 512)          787968    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                25650     
__________

In [49]:
batch_size = 32
epochs = 4

In [50]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=1, \
                          verbose=1, mode='auto')

In [51]:
hist3 = model.fit(X_t,train_Y, batch_size=batch_size, epochs=epochs,validation_split=0.1,callbacks=[earlystop])

Train on 17080 samples, validate on 1898 samples
Epoch 1/4
17080/17080 [==============================] - 299s 18ms/step - loss: 0.6816 - acc: 0.5519 - f1: 0.6984 - val_loss: 0.7688 - val_acc: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/4
17080/17080 [==============================] - 298s 17ms/step - loss: 0.6678 - acc: 0.5631 - f1: 0.6871 - val_loss: 0.7400 - val_acc: 0.5664 - val_f1: 0.0000e+00
Epoch 3/4
17080/17080 [==============================] - 305s 18ms/step - loss: 0.6633 - acc: 0.5708 - f1: 0.6947 - val_loss: 0.6612 - val_acc: 0.8556 - val_f1: 0.0000e+00
Epoch 4/4
17080/17080 [==============================] - 301s 18ms/step - loss: 0.6577 - acc: 0.5799 - f1: 0.6929 - val_loss: 0.8837 - val_acc: 0.1307 - val_f1: 0.0000e+00
Epoch 00004: early stopping


In [ ]:
plot_model_history(hist3)

In [ ]:
model.evaluate(X_te,test_Y)

In [33]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


